# Verify the enviroment

In [1]:
import sys
sys.executable

'/Users/ukilucas/anaconda3/envs/tensorflow_cpu/bin/python'

In [2]:
from jupyter_core.paths import jupyter_data_dir
print(jupyter_data_dir())

/Users/ukilucas/Library/Jupyter


# Import Packages

In [3]:
# help("modules")
import tensorflow as tf
print (tf.__version__)

0.12.1


In [4]:
from alexnet import AlexNet

In [5]:
import sklearn
print (sklearn.__version__)
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

0.18.1


In [6]:
import pickle
import time


# Fetch (road signs) data from hard drive

In [7]:
nb_classes = 43
epochs = 10
batch_size = 24 # This epoch took: 20.488 minutes on MacBook Pro 16Gb CPU
#batch_size = 256 # 21 minutes on MacBook Pro 16Gb CPU

timer_start = time.time()
with open('../DATA/german_traffic_signs/train.p', 'rb') as f:
    data = pickle.load(f)
print("Time: %.3f seconds" % (time.time() - timer_start))

Time: 0.247 seconds


# Split data to have 1/3 test set

In [8]:
timer_start = time.time()

X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.33, random_state=0)

print("Time: %.3f seconds" % (time.time() - timer_start))

Time: 0.044 seconds


In [9]:
timer_start = time.time()

features = tf.placeholder(tf.float32, (None, 32, 32, 3))
labels = tf.placeholder(tf.int64, None)
features_227x227 = tf.image.resize_images(features, (227, 227))

print("Time: %.3f seconds" % (time.time() - timer_start))

Time: 0.018 seconds


In [10]:
# Returns the second final layer of the AlexNet model,
# this allows us to redo the last layer for the traffic signs
# model.
fc7 = AlexNet(features_227x227, feature_extract=True)

In [11]:
fc7 = tf.stop_gradient(fc7)

In [12]:
shape = (fc7.get_shape().as_list()[-1], nb_classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.global_variables_initializer()

preds = tf.arg_max(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

In [13]:
def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

In [14]:
saver = tf.train.Saver()
MODEL_NAME = '../MODELS/TensorFlow_feature_extraction_AlexNet_'
print("Model will be saved to " + MODEL_NAME)

Model will be saved to ../MODELS/TensorFlow_feature_extraction_AlexNet_


In [15]:
with tf.Session() as sess:
    sess.run(init_op)
    print("this will run for %d epochs, each epoch takes 20 minutes on 4 core i7" % epochs)
    for epoch in range(epochs):
        timer_start = time.time()
        # training
        X_train, y_train = shuffle(X_train, y_train)
        
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            sess.run(train_op, feed_dict={features: X_train[offset:end], labels: y_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_val, y_val, sess)
        print("Epoch", epoch + 1)
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("This epoch took: %.3f minutes" % ((time.time() - timer_start)/60) )
        step = epoch + 1 # zero based epochs
        saver.save(sess, MODEL_NAME + str(val_acc), global_step=step)
        print("")
        
# MacBook Pro i7 16GB, 8 CPU (hyper threaded 4 core)
# Epoch 10
# Validation Loss = 0.156711707159
# Validation Accuracy = 0.962361851886
# This epoch took: 21.596 minutes

this will run for 10 epochs, each epoch takes 20 minutes on 4 core i7
Epoch 1
Validation Loss = 0.323206424801
Validation Accuracy = 0.907411700027
This epoch took: 21.215 minutes

Epoch 2
Validation Loss = 0.246307476823
Validation Accuracy = 0.929901845876
This epoch took: 20.488 minutes



KeyboardInterrupt: 